# Quadrature Phase Modulation

In [ ]:
from disiple.signals import TimeSignal, Spectrum
import numpy as np
from bokeh.plotting import show
from bokeh.layouts import row, column, gridplot

In [ ]:
samples_per_bit = 1000
bit_interval = 1e-9 # [seconds per bit]
bit_rate = 1 / bit_interval # [bits per second]
sample_rate = samples_per_bit * bit_rate # [samples per second=Hz]

### Message Signal

In [ ]:
bit_stream = [0, 0, 0, 1, 1, 0, 1, 1, 0]
samples_mod = np.repeat(bit_stream, samples_per_bit)
mod_signal = TimeSignal(samples_mod, sample_rate)
mod_spec = Spectrum.from_timesignal(mod_signal, dB=False)

### Split Message Signal in Two

In [ ]:
padded_bit_stream = np.concatenate((bit_stream, [0]*(len(bit_stream) % 2)))

in_phase_stream = padded_bit_stream[1::2]
samples_in_phase = np.repeat(in_phase_stream, samples_per_bit)
in_phase_signal = TimeSignal(samples_in_phase, sample_rate)
in_phase_spec = Spectrum.from_timesignal(in_phase_signal, dB=False)

quad_phase_stream = padded_bit_stream[0::2]
samples_quad_phase = np.repeat(quad_phase_stream, samples_per_bit)
quad_phase_signal = TimeSignal(samples_quad_phase, sample_rate)
quad_phase_spec = Spectrum.from_timesignal(quad_phase_signal, dB=False)

### Two Carrier Signals

In [ ]:
amp_carr = 1
freq_carr = 3 * bit_rate
qpsk_times = np.arange(0, len(in_phase_stream) * bit_interval, 1 / sample_rate)

samples_in_carr = amp_carr * np.sin(2*np.pi*freq_carr*qpsk_times)
in_carr_signal = TimeSignal(samples_in_carr, sample_rate)
in_carr_spec = Spectrum.from_timesignal(in_carr_signal, dB=False)

samples_quad_carr = amp_carr * np.cos(2*np.pi*freq_carr*qpsk_times)
quad_carr_signal = TimeSignal(samples_quad_carr, sample_rate)
quad_carr_spec = Spectrum.from_timesignal(quad_carr_signal, dB=False)

### Quadrature Phase Shift Keying

In [ ]:
samples_in_bpsk = np.where(samples_in_phase == 1, samples_in_carr, -samples_in_carr)
in_bpsk_signal = TimeSignal(samples_in_bpsk, sample_rate)
in_bpsk_spec = Spectrum.from_timesignal(in_bpsk_signal, dB=False)

samples_quad_bpsk = np.where(samples_quad_phase == 1, samples_quad_carr, -samples_quad_carr)
quad_bpsk_signal = TimeSignal(samples_quad_bpsk, sample_rate)
quad_bpsk_spec = Spectrum.from_timesignal(quad_bpsk_signal, dB=False)

qpsk_signal = in_bpsk_signal + quad_bpsk_signal
qpsk_spec = Spectrum.from_timesignal(qpsk_signal, dB=False)

### Create Figures

In [ ]:
mag_range = (0, amp_carr/2 * 1.05)
freq_range = (0, 1.5e10)

mod_fig = mod_signal.plot(title='Message Signal', active_inspect=None)
mod_spec_fig = mod_spec.plot(title='Magnitude Spectrum of Message Signal', x_range=freq_range, y_range=mag_range, active_inspect=None)

in_phase_fig = in_phase_signal.plot(title='In-Phase Message Signal', active_inspect=None, line_color='darkseagreen')
in_phase_spec_fig = in_phase_spec.plot(title='Magnitude Spectrum of In-Phase Message Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkseagreen')
quad_phase_fig = quad_phase_signal.plot(title='Quadrature-Phase Message Signal', active_inspect=None, line_color='indianred')
quad_phase_spec_fig = quad_phase_spec.plot(title='Magnitude Spectrum of Quadrature-Phase Message Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='indianred')

in_carr_fig = in_carr_signal.plot(title='In-Phase Carrier Signal', active_inspect=None, line_color='darkolivegreen')
in_carr_spec_fig = in_carr_spec.plot(title='Magnitude Spectrum of In-Phase Carrier Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkolivegreen')
quad_carr_fig = quad_carr_signal.plot(title='Quadrature-Phase Carrier Signal', active_inspect=None, line_color='crimson')
quad_carr_spec_fig = quad_carr_spec.plot(title='Magnitude Spectrum of Quadrature-Phase Carrier Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='crimson')

in_bpsk_fig = in_bpsk_signal.plot(title='In-Phase BPSK Signal', active_inspect=None, line_color='darkgreen')
in_bpsk_spec_fig = in_bpsk_spec.plot(title='Magnitude Spectrum of In-Phase BPSK Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkgreen')
quad_bpsk_fig = quad_bpsk_signal.plot(title='Quadrature-Phase BPSK Signal', active_inspect=None, line_color='darkred')
quad_bpsk_spec_fig = quad_bpsk_spec.plot(title='Magnitude Spectrum of Quadrature-Phase BPSK Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkred')

qpsk_fig = qpsk_signal.plot(title='QPSK Signal', active_inspect=None, line_color='darkgoldenrod')
qpsk_spec_fig = qpsk_spec.plot(title='Magnitude Spectrum of QPSK Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkgoldenrod')

### Link Frequency Axes

In [ ]:
from itertools import product
def link_x_axes(figs):
    for fig1, fig2 in product(figs, figs):
        fig1.x_range.js_link('start', fig2.x_range, 'start')
        fig1.x_range.js_link('end', fig2.x_range, 'end')
link_x_axes({mod_spec_fig, in_phase_spec_fig, quad_phase_spec_fig, in_carr_spec_fig, quad_carr_spec_fig, in_bpsk_spec_fig, quad_bpsk_spec_fig, qpsk_spec_fig})

### Display Figures

In [ ]:
width = 700
plot = column(
    row(mod_fig, mod_spec_fig, width=width),
    row(
        gridplot([in_phase_fig, quad_phase_fig, in_carr_fig, quad_carr_fig, in_bpsk_fig, quad_bpsk_fig], ncols=2, width=width//2), 
        gridplot([in_phase_spec_fig, quad_phase_spec_fig, in_carr_spec_fig, quad_carr_spec_fig, in_bpsk_spec_fig, quad_bpsk_spec_fig], ncols=2, width=width//2),
    ),
    row(qpsk_fig, qpsk_spec_fig, width=width),
)
show(plot)

In [ ]:
from bokeh.plotting import save
from bokeh.resources import INLINE
save(plot, filename='qpsk.html', title='Quadrature Phase Shift Keying', resources=INLINE)